In [1]:
import pandas as pd
import mibian as mb
import yfinance as yf
from datetime import datetime

/Users/jteruya/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
ticker = yf.Ticker("CCL")

In [3]:
def generate_black_scholes_call_chain_input(ticker, expiration_date, interest_rate):
    df = pd.DataFrame(ticker.option_chain(expiration_date).calls[['strike', 'lastPrice']])
    df['underlying_price'] = ticker.history(period="0d").iloc[0]['Close']
    df['interest_rate'] = interest_rate
    df['days_to_expiration'] = (datetime.strptime(expiration_date, '%Y-%m-%d') - datetime.now()).days + 1
    return df

In [4]:
df = generate_black_scholes_call_chain_input(ticker, '2020-06-18', 3)
df.head(10)

,strike,lastPrice,underlying_price,interest_rate,days_to_expiration
0,2.5,20.03,20.42,3,1
1,5.0,16.10,20.42,3,1
2,6.5,7.70,20.42,3,1
3,7.5,12.00,20.42,3,1
4,8.0,8.85,20.42,3,1
5,8.5,9.50,20.42,3,1
6,9.5,12.68,20.42,3,1
7,10.0,10.70,20.42,3,1
8,10.5,11.10,20.42,3,1
9,11.0,11.05,20.42,3,1


In [5]:
## Run quick test...
c = mb.BS([20.42, 2.5, 3, 1], callPrice=20.03)
c.impliedVolatility

500.0